In [3]:
import numpy as np
import os
import sys
from typing import List, Tuple, Optional

from tinytorch.core.tensor import Tensor
from tinytorch.core.autograd import Function, AddBackward, MulBackward, MatmulBackward, enable_autograd

enable_autograd()

## Unit Test: Function Classes

In [4]:
def test_unit_function_class():
    print("🔬 Unit Test: Function Classes...")

    a = Tensor([1, 2, 3], requires_grad= True)
    b = Tensor([4, 5, 6], requires_grad= True)

    # Test AddBackward
    add_func = AddBackward(a, b)
    grad_output = np.array([1, 1, 1])
    grad_a, grad_b = add_func.apply(grad_output)
    assert np.allclose(grad_a, grad_output), f'AddBackward grad_a fialed: {grad_a}'
    assert np.allclose(grad_b, grad_output), f'AddBackward grad_b failed: {grad_b}'

    # Test MulBackward
    mul_func = MulBackward(a, b)
    grad_a, grad_b = mul_func.apply(grad_output)
    assert np.allclose(grad_a, b.data), f"MulBackward grad_a failed: {grad_a}"
    assert np.allclose(grad_b, a.data), f"MulBackward grad_b failed: {grad_b}"

    # Test MatmulBackward
    a_mat = Tensor([[1, 2], [3, 4]], requires_grad= True)
    b_mat = Tensor([[5, 6], [7, 8]], requires_grad= True)
    matmul_func = MatmulBackward(a_mat, b_mat)
    grad_output = np.ones((2, 2))
    grad_a, grad_b = matmul_func.apply(grad_output)
    assert grad_a.shape == a_mat.shape, f"MatmulBackward grad_a shape: {grad_a.shape}"
    assert grad_b.shape == b_mat.shape, f"MatmulBackward grad_b shape: {grad_b.shape}"
    
    
    print("✅ Function classes work correctly!")

if __name__=='__main__':
    test_unit_function_class()

🔬 Unit Test: Function Classes...
✅ Function classes work correctly!


## Unit Test: Autograd

In [3]:
def test_unit_tensor_autograd():
    print("🔬 Unit Test: Tensor Autograd Enhancement...")

    # Test simple gradient computation 
    x = Tensor([2.0], requires_grad= True)
    y = x * 3
    z = y + 1 # z = 3x + 1
    j = z - 1

    z.backward()
    assert np.allclose(x.grad, [3.0]), f'Expected [3.0], got {x.grad}'

    # Test matrix multiplication gradients
    a = Tensor([[1.0, 2.0]], requires_grad= True) # 1x2
    b = Tensor([[3.0], [4.0]], requires_grad= True) # 2x1
    c = a.matmul(b)

    c.backward()
    assert np.allclose(a.grad, [[3.0, 4.0]]), f'Expected [[3.0, 4.0]] ,got {a.grad}'
    assert np.allclose(b.grad, [[1.0], [2.0]]), f'Expected [[1.0], [2.0]], got {b.grad}'

    # Test computation graph with multiple operations
    x = Tensor([1.0, 2.0], requires_grad= True)
    y = x * 2
    z = y.sum()


    
    z.backward()
    assert np.allclose(x.grad, [2.0, 2.0]), f'Expected [2.0, 2.0], got {x.data}'



    print("✅ Tensor autograd enhancement works correctly!")

if __name__=='__main__':
    test_unit_tensor_autograd()

🔬 Unit Test: Tensor Autograd Enhancement...
✅ Tensor autograd enhancement works correctly!


## Module Testing: Automatic differentation

In [4]:
def test_module():
    print("🧪 RUNNING MODULE INTEGRATION TEST")
    print("="*50)

    # running all unit tests
    test_unit_function_class()
    # test_unit_tensor_autograd()

    # Multilayer computation graph
    print('Integration Test Multi-layer Neural Network...')
    print("\nRunning integration scenarios...")

    # create a 3-layer computation: x -> Linear -> Linear -> Loss
    x = Tensor([[1.0, 2.0]], requires_grad= True)
    W1 = Tensor([[0.5, 0.3, 0.1], [0.2, 0.4, 0.6]], requires_grad= True)
    b1 = Tensor([[0.1, 0.2, 0.2]], requires_grad= True)

    # First layer
    h1 = x.matmul(W1) + b1
    assert h1.shape == (1, 3)
    assert h1.requires_grad == True

    # Second layer
    W2 = Tensor([[0.1], [0.2], [0.3]], requires_grad= True)
    h2 = h1.matmul(W2)
    assert h2.shape == (1, 1)

    # compute simple loss
    loss = h2 * h2
    # backward pass
    loss.backward()

    # verify all parameters have gradients
    assert x.grad is not None
    assert W1.grad is not None
    assert b1.grad is not None
    assert W2.grad is not None
    assert x.grad.shape == x.shape
    assert W1.grad.shape == W1.shape
    print("✅ Multi layer neural network works")

    # Test gradient accumulation
    print("🔬 Integration Test: Gradient Accumulation...")
    x = Tensor([2.0], requires_grad= True)

    # First computation 
    y1 = x * 3
    y1.backward()
    first_grad = x.grad.copy()

    # second computation (should accumulate)
    y2 = x*5
    y2.backward()

    assert np.allclose(x.grad, first_grad + 5.0), f"Gradients should accumulate"
    print("✅ Gradient accumulation works")

    print("🔬 Integration test complex operations")
    

    # test complex mathematical computations
    a = Tensor([[1.0, 2.0], [3.0, 4.0]], requires_grad= True)
    b = Tensor([[2.0, 1.0], [1.0, 2.0]], requires_grad= True)

    # complex computations ((a @ b )+ a) *b
    temp1 = a.matmul(b)
    temp2  = temp1 + a
    result = temp2 * b
    final = result.sum()

    final.backward()

    assert a.grad is not None
    assert b.grad is not None
    assert a.grad.shape == a.shape
    assert b.grad.shape == b.shape
    
    print("✅ Complex mathematical operations work!")

    print("\n" + "=" * 50)
    print("🎉 ALL TESTS PASSED! Module ready for export.")
    print("Run: tito module complete 06_autograd")

if __name__=='__main__':
    test_module()

🧪 RUNNING MODULE INTEGRATION TEST
🔬 Unit Test: Function Classes...
✅ Function classes work correctly!
Integration Test Multi-layer Neural Network...

Running integration scenarios...
✅ Multi layer neural network works
🔬 Integration Test: Gradient Accumulation...
✅ Gradient accumulation works
🔬 Integration test complex operations
✅ Complex mathematical operations work!

🎉 ALL TESTS PASSED! Module ready for export.
Run: tito module complete 06_autograd
